In [ ]:
import numpy as np
from decision_trees import *

In [ ]:
class Node:
    
    def __init__(self, bool_leaf = False, threshold = None, feature = None, output = None):
        self.bool_leaf = bool_leaf
        self.threshold = threshold
        self.feature = feature
        self.output = output
    

In [ ]:
class gradient_boosting:
    
    def __init__(self, learning_rate, num_trees, max_depth, type = "Classification"):
        """
        desc : Constructor for gradient_boosting.

        learning_rate : (float) specifies the learning rate for our GB.
        num_trees : (int) number of trees in our decision tree.
        max_depth : (int) maximum depth of our decision tree.
        type : (string) from : ['Classification','Regression']. specifies the type of problem.
        
        return : (None)
        """
        self.learning_rate = learning_rate
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.list_trees = []
        self.type = type
    
    def build(self, X, Y):
        """
        desc : Adds new decision trees to self.list_trees

        X : (numpy) dataset we have to classify without the target variable.
        Y : (list) target variable.
        
        return : (None) Adds new decision trees to self.list_trees.
        """
        # Here we add initial guess onto our list.
        initial_guess = 0
        if self.type == 'Classification':
            initial_guess = np.log(sum(Y)/(len(Y)-sum(Y)))
        else:
            initial_guess = sum(Y)/len(Y)
        self.list_trees.append(initial_guess)
        
        # initialising prediction list
        pred = [initial_guess for i in Y]
        
        # Now we add further decision trees onto our list.
        for i in range(self.num_trees):
            # step 2(A) find the residuals
            residuals = Y - pred
            # step 2(B) build dec_tree and step 2(C) output of each leaf_node saved in itself.
            t = decision_tree.build_tree(X, self.max_depth, residuals, self.type)
            self.list_trees.append(t)
            # step 2(D) update pred list
            pred = pred + self.learning_rate*t.output(X)